In [1]:
import torch
import numpy as np
from torch import nn
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_epochs = 1
img_size = 224
batch_size = 64
num_classes = 10
lr = 3e-4
T = transforms.Compose(
    [
     transforms.Resize((img_size, img_size)),
     transforms.ToTensor()
    ]
)
print(device)

cuda


In [3]:
train_data = datasets.CIFAR10("data/", train=True, download=True, transform=T)
val_data = datasets.CIFAR10("data/", train=False, download=True, transform=T)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)
x, y = next(iter(train_loader))
print(len(train_data), x.shape, y.shape)

Files already downloaded and verified
Files already downloaded and verified
50000 torch.Size([64, 3, 224, 224]) torch.Size([64])


In [4]:
net = models.resnet18(pretrained=False)
net.fc = nn.Linear(net.fc.in_features, num_classes)
net.to(device)
inp = torch.randn(1, 3, 224, 224).to(device)
out = net(inp)
print(out.shape)
del inp, out

torch.Size([1, 10])


In [5]:
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
loss_fn = nn.CrossEntropyLoss()
def get_accuracy(preds, y):
    preds = preds.argmax(dim=1, keepdim=True)
    correct = preds.squeeze(1).eq(y)
    acc = correct.sum() / torch.FloatTensor([y.shape[0]]).to(device)
    return acc

In [6]:
def loop(net, loader, is_train):
    net.train(is_train)
    losses = []
    accs = []
    pbar = tqdm(loader, total=len(loader))
    for x, y in pbar:
        x = x.to(device)
        y = y.to(device)
        with torch.set_grad_enabled(is_train):
            preds = net(x)
            loss = loss_fn(preds, y)
            acc = get_accuracy(preds, y)
            losses.append(loss.item())
            accs.append(acc.item())
        if is_train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        pbar.set_description(f'epoch={epoch}, train={int(is_train)}')
        pbar.set_postfix(loss=f'{np.mean(losses):.4f}', acc=f'{np.mean(accs):.4f}')

In [7]:
for epoch in range(n_epochs):
    loop(net, train_loader, True)
    loop(net, val_loader, False)

epoch=0, train=0: 100%|██████████| 157/157 [00:15<00:00, 10.08it/s, acc=0.6428, loss=1.0020]
